<a href="https://colab.research.google.com/github/GusvdW/Predicting-Human-Trafficking-Incidents/blob/main/Project_3_Predicting_Human_Trafficking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import spacy
!pip install vaderSentiment
!pip install transformers
!pip install torch
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import BertTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from google.colab import files
from transformers import BertModel
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad
from imblearn.over_sampling import RandomOverSampler
!pip install --upgrade tweepy
import tweepy
import os
import re
from transformers import BertTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading dataset
uploaded = files.upload()
global_data = pd.read_csv('The Global Dataset 14 Apr 2020.csv')


Saving The Global Dataset 14 Apr 2020.csv to The Global Dataset 14 Apr 2020 (1).csv


<ipython-input-65-2ec9d976297e>:3: DtypeWarning: Columns (6,55,57) have mixed types. Specify dtype option on import or set low_memory=False.
  global_data = pd.read_csv('The Global Dataset 14 Apr 2020.csv')


In [ ]:
# Drop columns with excessive missing values or irrelevant for the model
global_data.drop(['Datasource', 'yearOfRegistration', 'citizenship', 'typeOfExploitConcatenated'], axis=1, inplace=True)

In [ ]:
# Handling missing values by imputing numeric columns with the median and categorical columns with the mode
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

In [ ]:
# Define a mapping from age ranges to numerical values (the midpoint of each range)
age_mapping = {
    '0--8': 4,
    '9--17': 13,
    '18--20': 19,
    '21--23': 22,
    '24--26': 25,
    '27--29': 28,
    '30--38': 34,
    '39--47': 43,
    '48+': 50
}

In [ ]:
# Apply the mapping to the 'ageBroad' column
global_data['ageBroad_numeric'] = global_data['ageBroad'].map(age_mapping)

In [ ]:
# Impute missing values in numerical columns
global_data['ageBroad_numeric'].fillna(global_data['ageBroad_numeric'].median(), inplace=True)

<ipython-input-70-23b3f037a9fd>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  global_data['ageBroad_numeric'].fillna(global_data['ageBroad_numeric'].median(), inplace=True)


In [ ]:
# Drop the original age column
global_data.drop(columns=['ageBroad'], inplace=True)

In [ ]:
# Apply LabelEncoder to object columns
object_cols = ['gender', 'majorityStatus', 'majorityStatusAtExploit', 'majorityEntry',
               'meansOfControlConcatenated', 'typeOfLabourConcatenated', 'typeOfSexConcatenated',
               'RecruiterRelationship', 'CountryOfExploitation']
label_encoder = LabelEncoder()
for col in object_cols:
    global_data[col] = label_encoder.fit_transform(global_data[col].astype(str))

In [ ]:
# Drop the target column from global_data
X_global_data = global_data.drop(columns=['isAbduction'])  # Features
y_global_data = global_data['isAbduction']  # Target

In [ ]:
# Define numerical and categorical columns for the preprocessor
numerical_cols = ['recruiterRelationFriend', 'recruiterRelationFamily', 'recruiterRelationOther',
                  'recruiterRelationUnknown', 'ageBroad_numeric']
categorical_cols = ['majorityStatus']

In [ ]:
# Preprocess the features: scale numerical and one-hot encode categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)  # One-hot encode categorical columns
    ]
)


In [ ]:
# Apply the preprocessor to the feature data
X_global_data_encoded = preprocessor.fit_transform(X_global_data)

In [ ]:
# Convert to float32 for compatibility with PyTorch
X_global_data_encoded = X_global_data_encoded.astype(np.float32)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_global_data_encoded, y_global_data, test_size=0.2, random_state=42)

In [ ]:
# Split into training and test sets
X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(X_global_data_encoded, y_global_data, test_size=0.2, random_state=42)



In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# File paths to the credentials
bearer_token_path = '/content/drive/My Drive/bearer_token.txt'
api_key_path = '/content/drive/My Drive/api_key.txt'
api_secret_key_path = '/content/drive/My Drive/api_secret_key.txt'
access_token_path = '/content/drive/My Drive/access_token.txt'
access_token_secret_path = '/content/drive/My Drive/access_token_secret.txt'

In [ ]:
# Read the token from the file
with open(file_path, 'r') as file:
    bearer_token = file.read().strip()


In [ ]:
# Check if files exist and reading the tokens
if os.path.exists(bearer_token_path) and os.path.exists(api_key_path) and os.path.exists(api_secret_key_path) and os.path.exists(access_token_path) and os.path.exists(access_token_secret_path):
    with open(bearer_token_path, 'r') as file:
        bearer_token = file.read().strip()

    with open(api_key_path, 'r') as file:
        api_key = file.read().strip()

    with open(api_secret_key_path, 'r') as file:
        api_secret_key = file.read().strip()

    with open(access_token_path, 'r') as file:
        access_token = file.read().strip()

    with open(access_token_secret_path, 'r') as file:
        access_token_secret = file.read().strip()


    print(f"Bearer token length: {len(bearer_token)}")
    print(f"API key length: {len(api_key)}")
    print(f"API secret key length: {len(api_secret_key)}")
    print(f"Access token length: {len(access_token)}")
    print(f"Access token secret length: {len(access_token_secret)}")
else:
    print("One or more authentication files not found")

Bearer token length: 112
API key length: 25
API secret key length: 50
Access token length: 50
Access token secret length: 45


In [ ]:
# Initialize Tweepy Client with all required credentials
client = tweepy.Client(bearer_token=bearer_token,
                       consumer_key=api_key,
                       consumer_secret=api_secret_key,
                       access_token=access_token,
                       access_token_secret=access_token_secret)

# Test fetching user info to verify the token works
try:
    response = client.get_me()
    print(response)
except tweepy.errors.Unauthorized as e:
    print(f"Error during authentication: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Response(data=<User id=1843307415527170048 name=Gustav van der Westhuizen username=gus_der72850>, includes={}, errors=[], meta={})


In [ ]:
# Define preprocess text
def preprocess_text(text):
    # Remove URLs, mentions, special characters
    text = re.sub(r"http\S+|@\S+|[^a-zA-Z\s]", '', text)
    text = text.lower()  # Lowercase
    return text

In [ ]:
# Define search query
query = "human trafficking"
max_results = 100
tweet_fields = ["created_at", "text", "author_id"]

try:
    # Fetch recent tweets based on the keyword
    response = client.search_recent_tweets(query=query, tweet_fields=tweet_fields, max_results=max_results)

    # Check if data is returned and preprocess the tweets
    conversations = []
    if response.data:
        for tweet in response.data:
            cleaned_text = preprocess_text(tweet.text)
            conversations.append(cleaned_text)

    # Print the collected and preprocessed tweets
    print(f"Collected {len(conversations)} tweets on '{query}'")
    for conv in conversations[:5]:  # Print the first 5 tweets for review
        print(conv)

except tweepy.errors.Unauthorized as e:
    print(f"Error during authentication: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Collected 100 tweets on 'human trafficking'
rt   israel is a terrorist state 

devil worshippers 

masters of deception 

its time to kick satan back down
rt  just your monthly reminder im a proud gay american human trafficking survivor amp gen z supporting  thi
check out leonreeds video tiktok   are all our politicians complicit in this blatant human trafficking
rt  i once asked a florida state prosecutor to name the biggest racket in his steamy state i guessed aloud that he would give
rt  san antonio police arrested  gang members including  tren de aragua venezualan gang members who took over a nearly 


In [ ]:
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Tokenize the collected conversations for BERT
if conversations:
    tokenized_conversations = tokenizer(conversations, padding=True, truncation=True, return_tensors="pt")
    print(f"Tokenized conversations shape: {tokenized_conversations['input_ids'].shape}")

Tokenized conversations shape: torch.Size([100, 59])


In [ ]:
# Preprocess the conversations
preprocessed_conversations = [preprocess_text(conv) for conv in conversations]

In [ ]:
# Tokenize the conversations for BERT
tokenized_conversations = tokenizer(preprocessed_conversations, padding=True, truncation=True, return_tensors="pt")

In [ ]:
# Random Forest model
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train_global, y_train_global)

RandomForestClassifier(random_state=42)

In [ ]:
# Evaluate on test data
y_pred_global = model_rf.predict(X_test_global)

In [ ]:
# BERT model
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Check the number of samples in structured data
print(f"Shape of X_train_global: {X_train_global.shape}")

Shape of X_train_global: (39040, 7)


In [ ]:
print(f"Number of conversation samples: {len(conversations)}")

Number of conversation samples: 100


In [ ]:
# Pass tokenized conversations through BERT to get features
with torch.no_grad():
    outputs = bert_model(**tokenized_conversations)
    bert_features = outputs.last_hidden_state.mean(dim=1)  # Take mean of last hidden state

In [ ]:
# Repeat the conversation samples to match the number of structured samples (39040)
repeated_bert_features = np.tile(bert_features.numpy(), (39040 // 100, 1))

In [ ]:
# Ensure the sizes match exactly by slicing the repeated data
repeated_bert_features = repeated_bert_features[:39040]

In [ ]:
# Number of samples in structured data and conversation data
num_structured_samples = X_train_global.shape[0]
num_conversation_samples = bert_features.shape[0]

# Calculate how many times to repeat the conversation samples
repeats = num_structured_samples // num_conversation_samples
remainder = num_structured_samples % num_conversation_samples

# Repeat the conversation features and then append the remainder
repeated_bert_features = np.tile(bert_features.numpy(), (repeats, 1))

# Add extra rows to match exactly 39040 if there's a remainder
if remainder > 0:
    repeated_bert_features = np.vstack((repeated_bert_features, bert_features.numpy()[:remainder]))

# Ensure the final size matches
print(f"Shape of repeated_bert_features: {repeated_bert_features.shape}")

# Combine the structured and unstructured features
X_combined_train = np.hstack((X_train_global, repeated_bert_features))

print(f"Shape of combined features: {X_combined_train.shape}")

Shape of repeated_bert_features: (39040, 768)
Shape of combined features: (39040, 775)


In [ ]:
# Number of samples in structured test data and conversation data
num_test_samples = X_test_global.shape[0]
num_conversation_samples = bert_features.shape[0]

# Calculate how many times to repeat the conversation samples
repeats = num_test_samples // num_conversation_samples
remainder = num_test_samples % num_conversation_samples

# Repeat the conversation features and then append the remainder for test data
repeated_bert_features_test = np.tile(bert_features.numpy(), (repeats, 1))

# Add extra rows to match exactly 9761 if there's a remainder
if remainder > 0:
    repeated_bert_features_test = np.vstack((repeated_bert_features_test, bert_features.numpy()[:remainder]))

# Ensure the final size matches
print(f"Shape of repeated_bert_features_test: {repeated_bert_features_test.shape}")

Shape of repeated_bert_features_test: (9761, 768)


In [ ]:
# Combine the structured and unstructured features for the test set
X_combined_test = np.hstack((X_test_global, repeated_bert_features_test))

print(f"Shape of combined features: {X_combined_test.shape}")

Shape of combined features: (9761, 775)


In [ ]:
# Predict using the final model
y_pred_final = final_model.predict(X_combined_test)

# Output predictions
print(f"Predictions shape: {y_pred_final.shape}")

Predictions shape: (9761,)


In [ ]:
# Assuming y_test_global is the true labels for your test data
accuracy = accuracy_score(y_test_global, y_pred_final)
precision = precision_score(y_test_global, y_pred_final, average='weighted')  # Use 'weighted' for multiclass
recall = recall_score(y_test_global, y_pred_final, average='weighted')        # Use 'weighted' for multiclass
f1 = f1_score(y_test_global, y_pred_final, average='weighted')                # Use 'weighted' for multiclass

print(f"Accuracy: {accuracy}")
print(f"Precision (weighted): {precision}")
print(f"Recall (weighted): {recall}")
print(f"F1 Score (weighted): {f1}")

Accuracy: 0.7894682921831779
Precision (weighted): 0.7841201538154805
Recall (weighted): 0.7894682921831779
F1 Score (weighted): 0.7851996776893104


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
